<a href="https://colab.research.google.com/github/Guo-bot-1998/Appendicitis/blob/master/Appendicitis_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [ ]:
!pip install timm

In [ ]:
import nibabel as nib
import numpy as np
import os
import pandas as pd
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
import timm

## 常用功能

In [ ]:


def read_data(directory, shift=0, termi=10) -> dict[str, np.array]:
    """Reads all the .nii files in a given directory and returns a dictionary.

    The dictionary keys are the file names without the .nii extension. The dictionary
    values are the corresponding numpy arrays.

    Args:
      directory: The directory containing the .nii files.
      shift: The number of files to skip at the beginning of the directory.
      termi: The number of files to be read in the directory.

    Returns:
      A dictionary of numpy arrays.
    """

    data_dict = {}
    k = 0
    ncut = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.nii'):
                if (k<shift)       :
                  k += 1
                  continue
                file_path = os.path.join(root, file)
                nii_file =  nib.load(file_path)
                data = nii_file.get_fdata()
                data_dict[file.strip('.nii')] = data
                ncut +=  1
                if ncut == termi:
                  return data_dict
    return data_dict



def read_label(excel_path) -> pd.DataFrame:
  """Reads a csv file containing ground-truth.
    The csv file should have two columns: 'id' and 'label'.
  """

  with open(excel_path, 'r') as f:
    df = pd.read_csv(f)
    df.set_index('id', inplace=True)
    return df


def read_submission(excel_path) -> pd.DataFrame:
  """Reads a csv file containing submission file.
    The csv file should follow the format given by Kaggle.
  """
  with open('submission.csv', 'r') as f:
    df = pd.read_csv(f)
    df.set_index('id', inplace=True)
    return df



def process_data(data:dict, labels:pd.DataFrame)\
 -> tuple[torch.tensor, torch.tensor, list[tuple[str, int]]]:
  """Process the data, get the id and number of each scan, and merge the images and labels.

  Args:
      data: The data of the scans
      labels: The labels of the scans

  Returns:
      A tuple of images, labels, and (id, # of cuts) of each scan
  """

  # 每個scan的id與數量
  scans_info = []
  # 圖片與對應label的tesnor
  images_list = []
  labels_list = []

  if dflabel.index.name != 'id':
    dflabel.set_index('id', inplace=True)
  for key, value in data.items():
    scans_info.append((key, value.shape[2]))
    scan  = dflabel.loc[dflabel.index.str.startswith(key+'_')]
    labels = np.array(scan['label'])

    images_t = torch.from_numpy(value).float().permute(2, 0, 1).unsqueeze(1)
    labels_t = torch.from_numpy(labels)
    images_list.append(images_t)
    labels_list.append(labels_t)

  # 合併
  images = torch.cat(images_list, dim=0)
  labels = torch.cat(labels_list, dim=0)
  print(f"read {len(scans_info)} scans")

  return images, labels.float(), scans_info


def argumenting(images, labels, n=5):
  where = (labels == True).nonzero(as_tuple=True)[0]
  bad_images = images[where]

  rep_imgs = bad_images.repeat(n, 1, 1, 1)
  rep_labels = torch.ones(rep_imgs.shape[0], dtype=labels.dtype)

  # import pdb
  # pdb.set_trace()
  # 隨機插入argumented圖片
  nimg = images.shape[0]
  rnd_pos = torch.randint(0, nimg, (rep_imgs.shape[0],))
  images = torch.cat((images, rep_imgs), dim=0)
  images = images[torch.argsort(torch.cat((torch.arange(nimg), rnd_pos)))]

  # 對應對置插入標籤
  labels = torch.cat((labels, rep_labels), dim=0)
  labels = labels[torch.argsort(torch.cat((torch.arange(nimg), rnd_pos)))]

  return images, labels


def get_confusion(guess, truth):
  right = guess == truth
  wrong = np.logical_not(right)
  TP = np.sum(np.logical_and(right, truth == np.ones(right.shape)))
  TN = np.sum(np.logical_and(right, truth == np.zeros(right.shape)))
  FN = np.sum(np.logical_and(wrong, truth == np.ones(wrong.shape)))
  FP = np.sum(np.logical_and(wrong, truth == np.zeros(wrong.shape)))
  return (TP,FP,FN,TN)



def print_results(prediction, labels):
  if torch.is_tensor(prediction):
    prediction = prediction.cpu().numpy()
  if torch.is_tensor(labels):
    labels = labels.cpu().numpy()

  TP,FP,FN,TN = get_confusion(prediction, labels)
  print(f"\n\
      真實值\n\
  預  +-----+-----+\n\
  測| TP: {TP}| FP: {FP}|\n\
  值| FN: {FN}| TN: {TN}|\n\
      +-----+-----+ \n")

  recall = TP/(TP + FN)
  precision = TP/(TP + FP)
  recall = 0 if np.isnan(recall) else recall.item()
  precision = 0 if np.isnan(precision) else precision.item()
  F1 = 0 if recall + precision == 0 else  (2*recall*precision/(recall+precision))

  print(f"{recall=}\n{precision=}\n{F1=}\n")



def custom_sort_key(val):
    parts = val.split('_')
    if len(parts) == 2 and parts[1].isdigit():
        return (parts[0], int(parts[1]))
    return (parts[0], -1)  # 使沒有_(數字)的id排最前面


def isgpu():
    """檢查是否有 CUDA 支持的 GPU"""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU is available")
    else:
        device = torch.device("cpu")
        # raise("GPU not available")
    return device


In [ ]:
device = isgpu()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 路徑


In [ ]:
cd /content/drive/MyDrive/AOCR2024

In [ ]:
!pwd

# 處理資料

In [ ]:
data = read_data("TrainValid_Image/train_data",termi=20,shift=6)
dflabel = read_label("TrainValid_ground_truth.csv")

In [ ]:
images,labels, _ = process_data(data,dflabel)

In [ ]:
images,labels = argumenting(images,labels,3)

In [ ]:
images.shape

# 訓練 (Unet with only encoder)

In [ ]:
class FC(nn.Module):
    def __init__(self, in_channels):
        super(FC, self).__init__()

        # Encoder部分
        self.encoder = nn.Sequential(
            nn.BatchNorm2d(1),  # Add BatchNorm after the first Conv2d
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),  # Add BatchNorm after the first Conv2d
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),  # Add BatchNorm after the second Conv2d
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),  # Add BatchNorm after the third Conv2d
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),  # Add BatchNorm after the fourth Conv2d
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Linear(128 * 128 * 256, 1)  # x 和 y 是經過 encoder 處理後的特徵圖尺寸


    def forward(self, x):
        x = self.encoder(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
model_name = "tf_efficientnetv2_m"
pretrained_model = timm.create_model(model_name, pretrained=True)

# 修改輸入通道
pretrained_model.conv_stem = nn.Conv2d(1, 24, kernel_size=3, stride=2, padding=1, bias=False)

# 修改輸出類別
num_classes = 1
pretrained_model.classifier = nn.Linear(pretrained_model.classifier.in_features, num_classes)

# 添加 Sigmoid 激活函數
pretrained_model = nn.Sequential(
    pretrained_model,
    nn.Sigmoid()
)

# 檢查模型結構
print(pretrained_model)


## 訓練 (EfficiencyNetV2_m)

In [ ]:
from torch.utils.data import Dataset, DataLoader
num_epochs = 10
batch_size = 6
lr = 0.01
num_batches = len(images) // batch_size  #最後data不滿一個batch丟棄
# 初始化模型、損失函數和優化器
# model = FC(in_channels=1).to(device)
model = pretrained_model


criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr)
model.to(device)

running_loss_list = []
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Assuming you have 'images' and 'labels' as your original data
dataset = CustomDataset(images, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_images, batch_labels in dataloader:
        batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_images)
        outputs = outputs.squeeze()
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    running_loss_list.append(running_loss/len(dataloader))
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}")

In [ ]:
num_epochs = 10
batch_size = 16
lr = 0.001
num_batches = len(images) // batch_size  #最後data不滿一個batch丟棄
# 初始化模型、損失函數和優化器
model = FC(in_channels=1).to(device)
# model = pretrained_model


criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr)

running_loss_list = []

# 訓練循環
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i in range(num_batches):
        batch_images = images[i*batch_size:(i+1)*batch_size].to(device)
        batch_labels = labels[i*batch_size:(i+1)*batch_size].to(device)

        optimizer.zero_grad()
        outputs = model(batch_images)
        outputs = outputs.squeeze()
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    running_loss_list.append(running_loss/num_batches)
    print(f"Epoch {epoch+1}, Loss: {running_loss/num_batches}")


# Loss圖

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(1,num_epochs+1)[:len(running_loss_list)], running_loss_list)
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# 儲存模型參數



In [ ]:
cd /content/drive/MyDrive/AOCR2024

In [ ]:
import json
appendix = input("請輸入要儲存的編號:")
modelname = model.__class__.__name__
filename = f"params/{modelname}/{modelname}{appendix}"
if not os.path.exists(os.path.dirname(filename)):
  os.mkdir(os.path.dirname(filename))

if os.path.isfile(filename+'.pth'):
    print(f"{filename}.pth exist.")
else:
    torch.save(model.state_dict(), f'{filename}.pth')

params = {
    'num_epochs': num_epochs,
    'batch_size': batch_size,
    'learning_rate': lr,
}

if os.path.isfile(filename+'.json'):
    print(f"{filename}.json exist.")
else:
    with open(f'{filename}.json', 'w') as f:
        json.dump(params, f)

# 讀取模型參數

In [ ]:
import json
remove_digits = str.maketrans('', '', '0123456789')
modelname = input("請輸入要獲取模型檔名:")
filename = f"params/{modelname.translate(remove_digits)}/{modelname}"

if 'model' not in globals() or model.__class__.__name__ != modelname.translate(remove_digits):
  exec(f'model = {modelname.translate(remove_digits)}(in_channels=1).to(device)')

if not os.path.isfile(filename+'.pth'):
    print(f"{filename}.pth not exist.")
else:
    print(model.load_state_dict(torch.load(filename+'.pth')))
with open(f'{filename}.json', 'r') as f:
    params = json.load(f)

# 評估


In [ ]:
data = read_data("TrainValid_Image/train_data",termi=4)
dflabel = read_label("TrainValid_ground_truth.csv")

In [ ]:
testimages,testlabels,info = process_data(data, dflabel)

In [ ]:
# 評估設置
num_epochs = params['num_epochs']
batch_size = params['batch_size']
num_batches = len(testimages) // batch_size  #最後data不滿一個batch丟棄

In [ ]:
model.eval()

predict_list = torch.tensor([]).to(device)
with torch.no_grad():  # 不更新梯度
    correct = 0
    total = 0
    for i in range(num_batches):
        batch_images = testimages[i*batch_size:(i+1)*batch_size].to(device)
        batch_labels = testlabels[i*batch_size:(i+1)*batch_size].to(device)

        outputs = model(batch_images)
        predicted = (outputs.squeeze() > 0.5).int()
        predict_list = torch.cat((predict_list,predicted),0)


predict_listq = predict_list.cpu()



In [ ]:
print_results(predict_listq, testlabels[:len(predict_list)])

# 輸出至submission.csv

In [ ]:
predict_list = predict_listq.numpy()

## 還原id與對應label，假設key按照scan輸入順序排列,每個key對應的scan的cuts數是nslice
output = {}
k = 0  #第幾個scan
ii = 0   #每個key輪到第幾個
id, nslice = info[k][0], info[k][1]
for i in range(len(predict_list)):

  if (ii >= nslice):
    #該換下一個scan了
    output[id] = int(predict_list[i-nslice:i].any()) #評估方式
    k += 1
    ii = 0
    id, nslice = info[k][0], info[k][1]

  label = predict_list[i]
  output[id+f'_{ii}'] = int(predict_list[i])
  ii += 1

output[id] = int(predict_list[i-ii:].any()) #補上最後一個scan評估

output = list(output.items())
dfout = pd.DataFrame(output)
dfout.columns = ['id', 'label']
dfout = dfout.sort_values(by='id', key=lambda x: x.map(custom_sort_key))
dfout.to_csv('submission.csv', index=False)

In [ ]:
dfout

# 其他

查看submission files

In [ ]:
dftest = read_submission('submission.csv')
dflabel = read_label("TrainValid_ground_truth.csv")

In [ ]:
import re

# 抓出scan-level data
pattern = re.compile(r'.*_[0-9]+$')  # 正則表達式匹配 "_數字" 結尾
mask = ~dftest.index.to_series().str.match(pattern)
scan_guess = np.array(dftest[mask]['label'])
scan_truth = np.array(dflabel.loc[dftest[mask]['label'].index]['label'])

mask = ~mask
cut_guess = np.array(dftest[mask]['label'])
cut_truth = np.array(dflabel.loc[dftest[mask]['label'].index]['label'])

In [ ]:
ss = 10
print("="*ss + "F1 score on scan level" + "="*ss)
print_results(scan_guess, scan_truth)
print(),print()
print("="*ss + "F1 score on cut level" +"="*ss )
print_results(cut_guess, cut_truth)